# 1. Optimization of deep learning parameters (딥러닝 파라미터의 최적화)
- 완전 연결층, 합성곱 신경망, LSTM층을 설계하고 구현하기 위해서는 다양한 파라미터의 값을 지정해야 함
1) 데이터 준비
2) 딥너링 모델 생성
3) 학습
4) 검증

In [1]:
# 데이터 준비
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers

data_no = pd.read_csv("ptbdb_normal.csv")
data_ab = pd.read_csv("ptbdb_abnormal.csv")
data_no = np.array(data_no)
data_ab = np.array(data_ab)

nTrain = 3000
nTest = 1000
X_train = np.concatenate((data_no[:nTrain, :], data_ab[:nTrain, :]), 0)
y_train = np.concatenate((np.zeros(nTrain, ), np.ones(nTrain, )), 0)
X_test = np.concatenate((data_no[nTrain:nTrain+nTest, :], data_ab[nTrain:nTrain+nTest, :]), 0)
y_test = np.concatenate((np.zeros(nTest, ), np.ones(nTest, )), 0)

y_test = to_categorical(y_test)
y_train = to_categorical(y_train)
X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)

In [2]:
# 딥러닝 모델 생성
model = Sequential()
model.add(layers.Conv1D(filters=16, kernel_size=3, input_shape=(X_train.shape[1], 1), activation="relu"))
model.add(layers.Conv1D(filters=16, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling1D(pool_size=3, strides=2))
model.add(layers.Conv1D(filters=32, kernel_size=3, input_shape=(X_train.shape[1], 1), activation="relu"))
model.add(layers.Conv1D(filters=32, kernel_size=3, activation="relu"))
model.add(layers.MaxPooling1D(pool_size=3, strides=2))
model.add(layers.LSTM(16))
model.add(layers.Dense(units=2, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(learning_rate=0.01), metrics=["accuracy"])

In [3]:
# 학습
model.fit(X_train, y_train, epochs=50, batch_size=128, validation_split=0.2)

Epoch 1/50


38/38 [==============================] - 3s 29ms/step - loss: 0.6377 - accuracy: 0.6448 - val_loss: 0.7925 - val_accuracy: 0.5342
Epoch 2/50
38/38 [==============================] - 1s 17ms/step - loss: 0.5880 - accuracy: 0.7006 - val_loss: 0.8043 - val_accuracy: 0.5283
Epoch 3/50
38/38 [==============================] - 1s 18ms/step - loss: 0.5764 - accuracy: 0.7204 - val_loss: 0.8151 - val_accuracy: 0.5283
Epoch 4/50
38/38 [==============================] - 1s 18ms/step - loss: 0.5795 - accuracy: 0.7129 - val_loss: 0.8449 - val_accuracy: 0.4633
Epoch 5/50
38/38 [==============================] - 1s 17ms/step - loss: 0.5683 - accuracy: 0.7188 - val_loss: 0.8360 - val_accuracy: 0.4925
Epoch 6/50
38/38 [==============================] - 1s 18ms/step - loss: 0.5553 - accuracy: 0.7240 - val_loss: 0.8443 - val_accuracy: 0.4817
Epoch 7/50
38/38 [==============================] - 1s 17ms/step - loss: 0.5598 - accuracy: 0.7256 - val_loss: 0.7960 - val_accuracy: 0.5258
Epoch 8/50


In [4]:
# 검증
o = model.predict(X_test)
o = np.argmax(o, 1)
y_test = np.argmax(y_test, 1)
sum(np.equal(y_test, o) / len(y_test))

63/63 [==============================] - 1s 9ms/step


0.9609999999999496

## 1-1. 합성곱층의 파라미터
- model.add(layers.Conv1D(filters=16, kernel_size=3, activation="relu"))
- model.add(layers.MaxPooling1D(pool_size=3, strides=2))
- 모델에 합성곱층과 풀링충을 추가하고 파라미터를 설정하는 코드 
- 간단한 코드이지만 합성 곱층은 LSTM 층이나 완전 연결층에 비해 설정해 주어야 할 부분이 비교적 많은 편
1) 필터 수
- filters=16
- 필터는 합성곱을 수행하는 가중치의 세트
- 필터의 개수가 많아질수록 보다 다양한 특징을 추출할 수 있음
- 복잡한 데이터를 인식하기 위해서는 다양한 특징을 추출하여 정보량을 늘리는 것은 꼭 필요하기 때문에, 필터 수를 충분히 크게 지정할 필요가 있음
- 일반적으로 필터 수가 클수록 올바르게 학습된 네트워크의 성능은 향상되지만, 필터가 올바르게 작동하기 위한 필터 내부의 가중치들이 올바르게 학습되어야 하고, 이런 네트워크 학습이 더욱 어려워짐
- 이 문제를 해결하기 위한 기본 전략은 일단 과도하게 많은 수의 필터를 사용하여 학습을 시도한 후, 크기를 조금씩 줄여나가는 것
- 다른 전략은 여러 개의 합성곱층을 사용하면서, 필터 수를 조금씩 늘려가는 것
- 합성곱층에서 필터 수를 지정하는 코드의 예는 다음과 같음
- model.add(layers.Conv1D(filters=8, kernel_size=3, activation="relu"))
- model.add(layers.Conv1D(filters=16, kernel_size=3, activation="relu"))
- model.add(layers.Conv1D(filters=32, kernel_size=3, activation="relu"))
2) 커널 크기
- kernel_size = 3
- 커널 크기는 가중치 배열의 크기
- 커널 크기는 3(2차원인 경우 (3, 3))으로 쓰는 것이 보편적이긴 하지만, 상황에 따라 커널의 크기는 다양하게 바꿀 수 있음
- 커널 크기가 클수록 넓은 범위를 포괄할 수 있고, 작을수록 지엽적인 특징을 추출하는 필터가 만들어짐
- 일례로 커널 크기를 1로 설정하면, 출력 노드 1개 당 가중치의 개수가 하나로 고정되며, 출력은 단순히 공통의 가중치에 각 입력 노드의 값을 곱한 결과가 됨
3) 활성화 함수
- activation="relu"
- 활성화 함수는 각 층에서 발생하는 출력을 변형시키는 역할을 하며, 하이퍼볼릭 탄젠트 등은 출력을 특정 범위로 조정하기도 함
- 합성곱층과 완전 연결층에서 주로 사용하는 활성화 함수인 ReLU는 0보다 작은 출력을 모두 무시하는 역할을 함
- 이것은 부분적으로 딥러닝에서 사용되는 대부분의 데이터들이 0 또는 양수라는점에 기인하며, 가중치의 범위를 간접적으로 제한하여, 학습이 보다 쉽고 빠르게 이루어지게 하기 위한 것
- ReLU가 출력의 범위를 0 이상의 정수로 조정한다면, 0과 1 사이, -1과 1 사이의 값으로 출력을 조정하는 함수도 있음
- ReLU를 사용하면서도 비슷한 효과를 얻는 방법은 별도의 정규화 층을 추가하는 것
- 순환 신경망의 활성화 함수를 ReLU로 변경하여 성능을 향상시킬 수 있다는 의견이 있으나, 아직까지 Keras에서는 순환 신경망의 활성화 함수를 변경하는 방법을 별도로 제공하지는 않음
- Keras를 사용한 코딩에서 대부분의 활성화 함수들은 Activation 파라미터에 코드를 지정하여 사용할 수 있음
- 다만, Leaky ReLU는 고급 활성화 함수로 간주되어 별도의 네트워크층으로 구성하여야 함
- 활성화 함수를 별도의 층으로 구성할 때에는 합성곱층 등의 Activation 파라미터를 None으로 지정한 후, 해당 활성화 함수층을 추가
- 1차원 합성곱과 Leaky ReLU를 추가하는 코드는 다음과 같음
- model.add(layers.Conv1D(filters=16, kernel_size=3, activation=None))
- model.add(layers.LeakyReLU())
- 딥러닝 모델을 구성함에 있어 활성화 함수는 ReLU를 사용하는 것이 최근의 보편적인 흐름이며, 처음 모델을 구성할 때에는 ReLU를 사용하는 것이 최근의 보편적인 흐름이며, 처음 모델을 구성할 때에는 ReLU를 사용하여 모델을 구성하는 것을 추천
- 하지만 다른 딥러닝 파라미터와 마찬가지로, 여기에도 정답은 없고, 활성화 함수를 바꾸어 가며 학습/테스트를 진행하여 성능의 차이를 살펴봄
- model.add(layers.Conv1D(filters=32, kernel_size=3, activation="relu"))
4) 풀링층의 파라미터
- 풀링은 노드의 수, 즉 특징의 개수를 줄이기 위한 목적으로 사용
- 풀/stride의 크기가 클수록 특징의 개수가 대폭 줄기 때문에, 학습 난이도가 감소하지만, 동시에 정보 손실이 있을 수 있으므로 적절한 균형을 유지하도록하는 것이 중요
- 풀/stride의 크기는 네트워크의 깊이(개수)와도 관련이 있음
- 풀/stride의 크기가 크면, 네트워크층을 거쳐갈수록 입력 데이터의 크기가 빠르게 줄어들며, 반대의 경우에는 여러 층을 사용하여도 상대적으로 큰 크기를 유지할 수 있기 때문
- 예를 들어 16*16 크기의 이미지를 입력으로 받아, 풀/stride의 크기가 2인 풀링층을 세 번만 거치면 4개(2*2)의 노드가 남지만, 만약 크기가 4인 풀링층을 사용한다면 2개 이상의 풀링층은 사용할 수가 없게 됨
- 합성곱층과 풀링층이 하나의 세트로 사용되는 점을 생각해 보면, 이것은 네트워크를 구성함에 있어 제법 큰 제약이 됨
- 풀링층을 사용하면서도 여러 개의 합성곱층을 사용하는 '딥 네트워크'를 구성하기 위해, 합성곱층과 풀링층을 n:1의 비율로 구성하는 경우도 있음
- 합성곱 여러 개를 연속적으로 추가한 후, 풀링층을 한 번만 추가하는 것
- 다음은 2:1로 합성곱과 풀링층의 세트를 2개 연속으로 구성하는 코드
- model.add(layers.Conv1D(filters=32, kernel_size=3, activation="relu"))
- model.add(layers.Conv1D(filters=32, kernel_size=3, activation="relu"))
- model.add(layers.MaxPooling1D(pool_size=3, strides=2))
- model.add(layers.Conv1D(filters=32, kernel_size=3, activation="relu"))
- model.add(layers.Conv1D(filters=32, kernel_size=3, activation="relu"))
- model.add(layers.MaxPooling1D(pool_size=3, strides=2))
- 이미지 데이터를 인식하는 네트워크에서의 풀링은 MaxPooling이 사용되는데, 이것은 0에 가까울수록 정보가 적어지는 이미지 특성 때문
- MaxPooling을 사용하여 해당 풀에서 최댓값을 선택하므로 정보의 손실을 최소화하는 것
- 따라서 작은 값이 보다 중요한 정보를 지니고 있다면 MinPooling을, 모든 값이 중요한 상황이라면 성능을 높이기 위해 AveragePooling을 사용하여 테스트를 해 볼 필요가 있음
- MinPooling은 Keras에서는 지원하지 않으므로, 필요 시에는 입력 데이터를 반전하여 사용하거나 MinPooling을 자체적으로 구현해야 함
- Keras에서 제공하는 주요 풀링층은 Global 풀링층(입력받은 배치에 대해 전체 평균을 계산해 주는 층)과 Local 풀링층(일반적으로 사용)
- 풀링층의 또 다른 파라미터는 padding
- 이 파라미터의 값이 'valid'이면 패딩 없이 풀링을 수행하고, 'same'인 경우 stride가 1일 때를 기준으로 동일한 크기의 출력을 발생시키도록 제로-패딩을 한 후 결과를 계산
- 파라미터를 명시적으로 지정하지 않으면 기본값인 'valid'로 지정됨
- padding = "valid"

## 1-2. LSTM층의 파라미터
1) 노드 수
- LSTM층은 합성곱층에 비해 파라미터의 수가 적어, 파라미터의 지정이 비교적 단순
- LSTM에서 필수적인 파라미터는 '노드 수'임. LSTM의 노드 수는 완전 열결층에서의 노드 수와 유사한 개념으로, 네트워크층의 출력개수를 의미
- 노드 수가 증가할수록 네트워크 내의 가중치 수가 늘어나 보다 복잡한 패턴을 인식할 수 있으나, 학습의 난이도가 상승하여 데이터의 개수가 충분하지 않은 경우 네트워크가 올바르게 학습되지 않을 수 있음
- LSTM층에서 노드 수의 지정은 다음과 같은 코드를 사용
- layers.LSTM(16)
2) 출력 형태
- return_sequences = True
- 파라미터의 값이 True이면 시퀀스-투-시퀀스(seq2seq), False면 시퀀스-투-원(seq2one)의 형태로 사용
- 기본값은 False이므로 별도로 지정하지 않는 경우 seq2one으로 사용됨
- LSTM을 연속해서 연결하려면 return_sequences의 값을 True로 지정해서 데이터의 시퀀스 형태를 유지해주어야 함
- model.add(layers.LSTM(16, return_sequences=True))
- 두 개 이상의 LSTM층을 연속으로 사용하는 코드는 다음과 같음
- model.add(layers.LSTM(16, return_sequences=True))
- model.add(layers.LSTM(16, return_sequences=True))
- model.add(layers.LSTM(16))
3) 양방향 LSTM(Bidirectional)
- model.add(layers.Bidrectional(layers.LSTM(16)))
- Bidirectional 옵션은 데이터 시퀀스의 역순으로 입력되는 데이터를 학습하는 네트워크를 추가하는 기능
- 본래의 LSTM은 데이터의 시퀀스가 순차적으로 입력되어 변화하는 (순방향) 정보를 학습하지만, 양방향 LSTM의 경우에는 역순으로 입력되는 데이터를 학습하는 층(역방향층)을 추가한 후, 순방향 출력과 역방향 출력을 합쳐 층의 출력으로 삼음
- 두 출력을 합치는 것은 단순히 두 결과를 이어 붙이는 것이어서, 양방향 LSTM을 적용하면 LSTM의 출력 개수가 두배로 늘어남

## 1-3. 완전 연결층의 파라미터
- model.add(layers.Dense(units=2, activation="softmax))
- 완전 연결층의 파라미터는 unit(유닛 또는 노드)의 갯수, 활성화 함수의 종류
- 유닛 수는 출력 노드의 갯수를 의미하며, 활성화 함수는 앞서 살펴본 바와 동일
- 마지막 Dense층은 분류 모델을 사용할 때는 softmax 함수, 그 외의 경우는 sigmoid, relu 등을 사용하면 됨

## 1-4. 모델 생성(컴파일) 시의 파라미터
1) 학습률과 최적화 도구
- 딥러닝에서의 학습은 에지의 가중치를 조정하며 수행되고, 가중치의 조정은 출력 오차를 줄이는 방향으로 진행
- 즉, 학습의 핵심은 가중치를 조정하여 오차를 최소화 시키는 것인데, 이를 수행하는 라이브러리를 '최적화 도구'라고 부름
- 사실 하나의 데이터에 대해서만 학습을 수행한다면, 해당 데이터가 원하는 출력을 발생시키도록 가중치를 수정하는 것은 매우 쉬운 일이며 한번에 완료할 수도 있음
- 하지만 여러 개의 데이터 샘플이 있을 때, 특정 데이터에 대한 오차를 최소화시키도록 학습을 시켜버린다면, 다른 데이터들에 대한 오차는 반대로 상승할 가능성이 높음
- 이 때문에 '학습률'이라고 하는 개념이 등장했음
- 특정 데이터(배치)에 대한 오차를 최소화시키기위해 가중치를 1만큼 더해 주어야 한다면, 1을 바로 더하지 않고, 학습률을 곱해 더함
- 학습률이 0.1이라면, 학습률이 1인 경우에 비해 10배 느리게 학습을 진행하는 것이지만, 여러 데이터에 대해 안정적으로 학습을 진행하기 위해서는 학습률은 충분히 작아야 함
- 보통 0.001에서 0.01 사이의 값이 사용됨
- 최근까지 연구 개발된, 그리고 Keras에서 사용할 수 있는 최적화 도구는 매우 다양하지만, 대부분의 최적화 도구는 경사 하강법(Gradient Descent Algorithm)을 기본으로 사용
- 경사 하강법은 랜덤하게 주어진 가중치의 값에서 출발하여, 가중치를 변화시켰을 때 발생하는 손실(Loss)의 변화를 미리 계산하고, 손실을 줄이는 방향으로 가중치를 변화시키는 것
- 경사 하강법에서 가중치를 찾는 것은 여러 가지 고려해야 할 사항이 많으며, 이를 개선하기 위해 다양한 알고리즘들이 개발되어 있음
- Keras에서 제공하는 대표적인 최적화 도구들
- 추계적 경사 하강법(SGD, Stochastic Gradient Decent Algorithm) : 가중치의 개수가 증가할수록 주변의 모든 값들을 살펴보는 것은 계산시간의 문제를 발생시킴. 통계적(stochastic) 방법을 사용하여 계산량을 감소시키는 방법
- optimizer=optimizers.SGD(learning_rate=0.01)
- 제곱평균제곱근 전파(RMSProp, Root Mean Square Propagation) : 가중치를 변화시키는 방향을 유지하려는 힘(모멘트)을 고려하여, 가중치를 수정(이동)함
- optimizer=optimizers.RMSProp(learning_rate=0.01)
- 적응적 모멘트 추정(Adam, Adaptive Moment Estimation) : 모멘트의 상태에 따라 학습률을 자동으로 변화시킴
- optimizer=optimizers.Adam(learning_rate=0.01)
- 최적화 도구들 중 가장 보편적으로 쓰이는 Adam을 사용하도록 지정하는 코드는 다음과 같음
- optimizer=optimizers.Adam()
2) 손실함수
- loss="categorical_crossentropy"
- 손실함수는 최종 출력에서 발생하는 오차의 크기를 계산하는 방법

## 1-5. 학습 파라미터와 오버피팅(Overfitting)
1) 학습 파라미터
- model.fit(X_train, y_train, epochs=50, batch_size=128, validation_split=0.2)
- 학습 시에 조정해야 하는 기본적인 파라미터는 '에포크 수(epochs)', '배치 크기(batch_size)', '검증 데이터 비율(validation_split)'
- 에포크 수는 전체 데이터를 몇 번 반복해서 학습시킬 것인지를, 배치 크기는 몇 개의 데이터 샘플을 한 번에 학습시킬 것인지를, '검증 데이터 비율'은 주어진 데이터에 대해 학습/검증 데이터를 나누는 비율을 의미
2) 오버피팅
- 학습 파라미터들을 사용하여 얻고자 하는 것은 데이터를 설명하는 이상적인 모델이고, 이 모델을 찾는 작업을 피팅(Fitting)이라고 함 
- 언더피팅 : 아직 데이터에 대한 학습이 충분히 이루어지지 않았으며, 학습 시에 발생하는 손실값은 매우 큼. 네트워크 구조가 적합하다면, 학습이 진행되어 감에 따라 모델은 '이상적인 피팅' 상태로 변화해 감
- 오버피팅 : 학습시의 손실값은 매우 작지만 테스트 결과는 부정적일 수 있음
- 학습 파라미터의 조정은 네트워크를 어떻게 언더피팅과 오버피팅의 사이에 두게 할 것인가에 관한 문제임
- 에포크 수가 너무 적으면 언더피팅이, 에포크 수가 너무 많으면 오버피팅이 발생
- 배치 크기가 너무 크면, 전체 데이터에 대한 손실을 한 번에 계산하기 때문에 오버피팅이 발생할 가능성이 있음
- 반대로 배치 크기가 너무 작으면 언더피팅이 발생할 수도 있음
- 오버피팅을 방지하기 위해서는 네트워크의 크기(네트워크층의 수, 필터/노드 수)가 문제의 복잡도/데이터의 수에 비해 너무 크지 않아야 함
- 문제의 복잡도가 단순할수록, 데이터의 길이/채널 수가 작을 수록, 데이터 샘플의 수가 작을수록 네트워크의 크기는 더 작아야 함
- 데이터의 길이/채널 수가 큰 데이터여서 네트워크의 크기를 크게 만들어야 하지만, 데이터 샘플 수가 적어 오버피팅이 발생하는 경우라면 추가적인 방법을 고려해 볼 수 있음
3) 검증 데이터의 분리
- 검증 데이터를 별도로 나누는 것은 학습을 멈추는 시점을 결정하기 위한 것
- 테스트 데이터와는 별도로 학습 데이터 중에서 일부를 검증용 데이터로 따로 떼어 놓고, 나머지 데이터들만 가지고 학습을 진행
- 에포크가 끝날 때마다 (또는 n번의 에포크가 끝날 때마다) 검증 데이터를 사용해 오버피팅이 발생했는지 체크
- 만약 학습 시의 정확도는 계속 상승하는데, 검증 데이터를 사용하였을 때의 정확도가 지속적으로 하강한다면, 오버피팅이 발생했다고 간주할 수 있으며 오버피팅의 발생이 시작하는 시점에 학습을 멈추도록하여 오버피팅을 막을 수 있음
- 중요한 것은 전체 데이터를 학습-테스트로 나눈 후, 학습용 데이터에서 검증용 데이터를 따로 분리해 사용한다는 것
- 학습을 멈추는 시점을 결정하는 데에 검증용 데이터를 사용하기 때문에 (즉, 검증용 데이터도 학습에 사용된다고 볼 수 있으므로) 검증을 수행하는 경우에는 기존의 학습용 데이터에서 다시 분리하여 사용
4) 드롭아웃층(Dropout Layer)
- model.add(layers.Dropout(0.3))
- 케라스에서 드롭아웃층을 사용하는 코드. layers.Dropout()의 파라미터(0.3)는 드롭아웃층에서 연결을 끊을 노드의 비율
- 오버피팅을 방지하기 위해 가장 보편적으로 사용되는 방법은 네트워크층 내부의 노드/연결을 랜덤하게 제외시키고 학습시키는 것
- 이 방법을 통해 모델은 특정 노드의 값에 민감하지 않도록 학습되며 더욱 일반적인 모델을 찾는 방향으로 학습이 이루어지게 됨
- 드롭아웃층은 기본적으로 들어온 입력값을 변화시키지 않고 그대로 출력으로 내보내지만, 학습 시에는 입력과 출력 사이의 연결을 랜덤하게 끊어 해당 노드의 값이 결과에 영향을 끼치지 못하도록 함
- 연결이 끊어진 노드는 데이터의 전파/역전파 과정에서 무시됨
- 학습이 완료되고 테스트를 수행할 때에는 연결을 끊지 않고, 모든 노드들을 연결시켜 사용
- 딥러닝 모델의 마지막 완전 연결층에 드롭아웃을 적용하는 전체 코드
- model = Sequential()
- model.add(layers.Conv1D(filters=16, kernel_size=3, input_shape=(X_train.shape[1], 1), activation="relu"))
- model.add(layers.Conv1D(filters=16, kernel_size=3, activation="relu"))
- model.add(layers.MaxPooling1D(pool_size=3, strides=2))
- model.add(layers.Conv1D(filters=32, kernel_size=3, input_shape=(X_train.shape[1], 1), activation="relu"))
- model.add(layers.Conv1D(filters=32, kernel_size=3, activation="relu"))
- model.add(layers.MaxPooling1D(pool_size=3, strides=2))
- model.add(layers.LSTM(16))
- model.add(layers.Dropout(0.3))
- model.add(layers.Dense(units=2, activation="softmax"))
- model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(learning_rate=0.01), metrics=["accuracy"])
- 드롭아웃은 완전 연결층의 앞에서 사용하는 경우가 많으며, 경우에 따라 합성곱층의 앞에서 사용하기도 함
- 합성곱층의 경우에는 노드가 2차원 구조를 가지고 있으므로 이를 더욱 잘 활용할 수 있는 드롭블록(Drop Block)층을 사용하기도 함

## 1-6. 파라미터 최적화 전략
1) 목표 정확도 설정
- 파라미터의 최적화에 들어가기에 앞서, 달성하고자 하는 목표 정확도를 정하는 것이 바람직
- 딥러닝이 달성할 수 있는 정확도는 문제의 난이도, 데이터의 크기 등에 따라 달라지기 때문에 일괄적으로 얘기하기는 어렵지만, 해당 데이터의 분석/예측 등을 해당 분야의 전문가가 직접 수행했을 때의 정확도를 기준으로 삼는 것이 추천됨
- 데이터가 익숙하지 않아 전문가 수준의 정확도를 알기 어렵다면, 비슷한 유형의 데이터에 대한 인공지능의 인식 정확도를 검색해서 찾아보는 것도 좋은 방법
- 높은 수준의 정확도를 달성하는 것이 목표가 아니라면, 정확도의 하한선을 고려해 보는 것이 좋음
- 모델이 전혀 학습되지 않는 경우의 정확도를 기준으로 삼아, 이 기준보다 높은 정확도를 달성하는 것을 목표로 삼는 것
- 랜덤하게 출력이 나오는 경우를 계산하면 됨
- 2개 클래스의 분류 문제를 다루고 있다면 50%, 4개 클래스라면 25%임
2) 오버피팅
- 네트워크 구조를 처음 구성할 때에는 네트워크를 해당 데이터를 해결할 수 있는 문제보다 조금 더 크게 구성하는 것이 좋음
- 약간의 오버피팅을 감수하고 이 방식을 통해 목표 정확도의 달성 가능 여부를 유추할 수 있음
- 첫 구성에서 오버피팅이 발생하지 않앗다면, 오버피팅이 발생할 때까지 (학습 정확도는 계쏙 증가하지만, 검증 정확도는 감소할 때까지) 네트워크 층의 개수, 필터와 노드의 수 등을 증가시킴
- 최적화도구, 손실함수 등의 파라미터 조정을 통해 정확도의 개선을 꾀해볼 수도 있음
- 이 단계에서 만족할 만한 학습 결과를 달성하지 못한 경우라면, 데이터 전처리(정규화 등)나 데이터의 추가 수집 등을 통해 입력 데이터 자체를 개선할 수 잇는지를 살펴보아야 함
- 문제가 너무 어렵다면, 문제를 단순화시키는 것도 좋음
- 분류할 클래스의 수를 줄여서 테스트하거나, 학습이 어려울 것으로 예상하는 데이터를 미리 뺀 다음 학습해보는 등의 방법
- 이와 같은 방법을 통해 학습이 잘되지 않는 원인을 파악하여 대처 방안을 수립할 수 있음
3) 오버피팅의 극복
- 앞 단계에서 구성한 네트워크가 오버피팅되지 않고 학습이 이루어지도록 한다면, 비교적 높은 성능의 딥러닝 모델을 만들 수 있음
- 오버피팅의 방지를 위해 앞서 학습한 드롭아웃(Dropout) 레이어의 추가, 데이터 증강, 에포크 수의 감소, 네트워크층의 제거, 필터/노드 수의 축소, 배치 사이즈의 축소 등을 고려하고 파라미터를 조정하여 테스트해볼 수 있음
- 초반의 파라미터 조정 시에는 단위를 조금 크게 두고 조정하는 것이 좋으며, 오버피팅의 정도가 완화됨에 따라 미세조정하면 됨